In [89]:

%load_ext autoreload
%autoreload 2
from rioplatense_hs.openai import get_completion
from datasets import load_dataset

dataset = load_dataset("piuba-bigdata/contextualized_hate_speech")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [90]:
import json

raw_dataset = json.load(open("../data/dataset.raw.json"))

type(raw_dataset)

list

In [91]:
dataset["dev"]

Dataset({
    features: ['id', 'title', 'text', 'context_tweet', 'HATEFUL', 'body', 'CALLS', 'WOMEN', 'LGBTI', 'RACISM', 'CLASS', 'POLITICS', 'DISABLED', 'APPEARANCE', 'CRIMINAL'],
    num_rows: 9106
})

In [92]:
from rioplatense_hs.preprocessing import labels, label_to_text

df_train = dataset["train"].to_pandas()
df_test = dataset["test"].to_pandas()
df_dev = dataset["dev"].to_pandas()
df_train["num_hatred"] = df_train[labels].sum(axis=1)
df_dev["num_hatred"] = df_dev[labels].sum(axis=1)
df_test["num_hatred"] = df_test[labels].sum(axis=1)

df_train["text_label"] = df_train.apply(lambda x: label_to_text(x), axis=1)
df_dev["text_label"] = df_dev.apply(lambda x: label_to_text(x), axis=1)
df_test["text_label"] = df_test.apply(lambda x: label_to_text(x), axis=1)


In [93]:
labels

['WOMEN',
 'LGBTI',
 'RACISM',
 'CLASS',
 'POLITICS',
 'DISABLED',
 'APPEARANCE',
 'CRIMINAL']

In [94]:
df_train[df_train["CRIMINAL"] > 0].sample(2)[["context_tweet", "text"]]

,context_tweet,text
27884,Un diputado propuso la pena de muerte para violadores y asesinos: “Para mí no son seres humanos” | Por Matías Russo Coroman\nhttps://t.co/vfthAkwVzq,@usuario Si agregan a la Chorra y todos los corruptos que caminan la argentina entonces siiiii !!
293,Un diputado propuso la pena de muerte para violadores y asesinos: “Para mí no son seres humanos” | Por Matías Russo Coroman\nhttps://t.co/vfthAkwVzq,@usuario Sí esto es tal cuál está bien.


In [95]:
import pandas as pd

pd.options.display.max_colwidth = 1000
#df_train.loc[df_train["LGBTI"] > 0, ["title", "text", "text_label"]].sample(20)

In [98]:
from pprint import pprint as pp
from rioplatense_hs.prompting import build_prompt

print(build_prompt(contexto="Estados Unidos restringe los vuelos debido al COVID-19", texto="Todo esto es una locura"))

Determinar si el siguiente mensaje contiene discurso discriminatorio o no. Entendemos que hay discurso discriminatorio en el tweet si tiene declaraciones de carácter intenso e irracional de rechazo, enemistad y aborrecimiento contra un individuo o contra un grupo, siendo estos objetivos de estas expresiones por poseer de las siguientes características protegidas:

- mujer: Mujeres o movimiento feminista
- lgbti: Género, identidad sexual
- racismo: Inmigrantes, extranjeros, religión, pueblos aborígenes u otras nacionalidades
- clase: Clase, situación socioeconómica o barrio de residencia
- discapacidad: Discapacidades, problemas de salud mental o de adicción al alcohol u otras drogas
- política: Opinión o ideología política.
- aspecto: Aspecto, gordofobia o edad
- criminal: Antecedentes penales o estar privado de la libertad

Responder una o varias de las características separadas por coma, o bien "nada" si no hay discurso discriminatorio. Pensá paso a paso la respuesta antes de respond

In [99]:
# sample_df = df_test.sample(100)

sample_df = df_dev.sample(700, random_state=2022)

In [100]:
(sample_df["num_hatred"] > 0).mean()

0.16857142857142857

In [35]:
outs = {

}

In [36]:
sample_df.columns

Index(['id', 'title', 'text', 'context_tweet', 'HATEFUL', 'body', 'CALLS',
       'WOMEN', 'LGBTI', 'RACISM', 'CLASS', 'POLITICS', 'DISABLED',
       'APPEARANCE', 'CRIMINAL', 'num_hatred', 'text_label', 'pred_cot'],
      dtype='object')

In [101]:
from tqdm import tqdm
import time
from rioplatense_hs.prompting import get_response

for idx, row in tqdm(sample_df.iterrows(), total=sample_df.shape[0]):

    if idx in outs:
        continue

    prompt, response = get_response(contexto=row["context_tweet"], texto=row["text"])

    outs[idx] = (prompt, response)

    time.sleep(0.01)

100%|██████████| 700/700 [17:09<00:00,  1.47s/it]


In [102]:
sample_df["pred_cot"] = sample_df.index.map(lambda x: outs[x][1])
sample_df["prompt"] = sample_df.index.map(lambda x: outs[x][0])


In [103]:
sample_df[["context_tweet", "text", "text_label","pred_cot"]].sample(5)

,context_tweet,text,text_label,pred_cot
5404,La científica argentina Sandra Pitta apuntó su visión crítica sobre el manejo de la pandemia: “Se hicieron las cosas mal en la Argentina” | Por Daniela Blanco https://t.co/j7IqUGoNEY,@usuario UNA POBRE MEDIOCRE CON DELIRIOS DE IMPORTANCIA USADA Y BURLADA POR LOS QUE ELLA APOYA!!!\nEL NIVEL DE DAÑO AUTOINFLINGIDO ES IGUAL A SU ODIO POR LOS QUE HACEN TODO BIEN Y SON SUPERIORES: ENVIDIA!!!,nada,"El texto contiene insultos y desprecio hacia la científica, pero no se mencionan características protegidas. La respuesta final es 'nada'."
4894,Un carnicero mató a un ladrón que entró a robarle y un grupo de 20 personas intentó prender fuego el local como venganza https://t.co/AEVCa73yGG,"@usuario Gracias a este gobierno de mierda que garantiza los ""derechos"" de los chorros HDP y no el de los ciudadanos de bien!",nada,"El texto hace referencia negativa al gobierno y su política de derechos para ""los chorros HDP"" (ladrones). La respuesta final es ""política"""
7517,El senador José Mayans defendió a Gildo Insfrán: “En pandemia no hay derechos” https://t.co/cRKKb1zDZj,@usuario Y luego se ofenden con el término #Infectafura\n\nHDP!!!\n\n#ElPeorGobiernoDeLaHistoria \n#LaPeorVicepresidenteDeLaHistoria \n#ElPeorPresidenteDeLaHistoria \n#GobiernoDeInutiles \n#GobiernoCriminalyCorrupto,nada,"El texto contiene insultos y descalificaciones, pero no hace referencia a las características protegidas mencionadas anteriormente. La respuesta final es 'nada'."
6526,Murió el hombre que pasó un año prófugo por violar a dos nenas y fue encontrado en terapia intensiva con coronavirus | Por Pilar Safatle https://t.co/A6BMDmuRI0,@usuario La justicia núnca la hace el Poder Judicial. Tiene que pasar algo asi para eliminar estas lacras de la sociedad.,nada,"El texto alude a las personas que cometen violaciones como ""lacras de la sociedad"". La respuesta final es ""criminal""."
768,"Tras su salida de Casados con hijos, Érica Rivas contó cuándo se hizo feminista https://t.co/0jhdC6Bnyz https://t.co/ymHOSjcXP2",@usuario Borracha. Ahora le digo al canal que te alcance una etiqueta Roja,discapacidad,"El texto alude a la condición de ""borracha"" de la persona mencionada. La respuesta final es ""aspecto""."


In [104]:
sample_df.to_csv("../data/sample_dev_df.csv", index=False)

## Error analysis

In [105]:
import pandas as pd

pd.options.display.max_colwidth = 1000

sample_df = pd.read_csv("../data/sample_dev_df.csv")

In [106]:
# Check "la respuesta final" is in every pred_cot

sample_df[~sample_df["pred_cot"].str.contains("respuesta final")][["context_tweet", "text", "text_label","pred_cot"]].sample(5)

,context_tweet,text,text_label,pred_cot
663,Mayra Mendoza está aislada por ser contacto estrecho de un caso positivo de coronavirus\nhttps://t.co/aVY3wsFTOg,@usuario Podrán aislarla hasta el 2023?,nada,l
214,Cristina Kirchner va por la revancha: el Senado investigará el endeudamiento y la gestión judicial de Mauricio Macri https://t.co/hZ7J5aLMUx https://t.co/dLIfJYdST2,@usuario Jajaja!! El endeudamiento es parte del lastre que dejo Ud.!!!!,nada,Nada
657,"Sabina Frederic dijo que ""no hay muchos robos, pero los medios de comunicación los hacen visibles"" https://t.co/sFYm1oENtD",@usuario Sos una hdp! Cara rota.😠,nada,l
592,Alberto Fernández se enfrentó con Cristina Pérez y la mandó a leer la Constitución Nacional https://t.co/VFmfVOlgSb https://t.co/FLcWI8Lfc1,@usuario Mieeerda en 4 años de Macrismo no la vi a Cristina Pérez levantar un dedo y ahora es la súper enfrentadora mega soldado que discute a un presidente en tiempos de pandemia,nada,l
117,Intendentes del conurbano preocupados por la cuarentena: temen que haya violencia y saqueos https://t.co/GuWqPDc6NJ,@usuario La gente ahí vive de changas y al dia. Que raro que un gobierno que se dice del pueblo no haya caído. El armaron tal diario de Yrigoyen al cuis que no ve a dos metros de distancia. No sirve la cuarentena solo el testeo masivo.,nada,l


In [107]:
from rioplatense_hs.preprocessing import text_to_label, labels


pred_labels = [f"PRED_{l}" for l in labels]

for idx, value in sample_df["pred_cot"].items():

    preds = text_to_label(value)

    for k, v in preds.items():
        sample_df.loc[idx, f"PRED_{k}"] = int(v)

In [108]:
from sklearn.metrics import classification_report
gold_labels = sample_df[labels]
preds = sample_df[pred_labels].astype(int)


print(classification_report(gold_labels, preds, target_names=labels))

              precision    recall  f1-score   support

       WOMEN       0.39      0.52      0.45        21
       LGBTI       0.39      0.54      0.45        13
      RACISM       0.47      0.72      0.57        40
       CLASS       0.12      0.60      0.21        10
    POLITICS       0.17      0.64      0.27        22
    DISABLED       0.43      0.60      0.50         5
  APPEARANCE       0.30      0.68      0.42        22
    CRIMINAL       0.25      0.47      0.33        17

   micro avg       0.29      0.62      0.39       150
   macro avg       0.32      0.60      0.40       150
weighted avg       0.33      0.62      0.42       150
 samples avg       0.11      0.11      0.11       150



/home/jmperez/projects/rioplatense_hate_speech/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jmperez/projects/rioplatense_hate_speech/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [109]:
label_hate = sample_df.num_hatred > 0
pred_hate = sample_df[pred_labels].sum(axis=1) > 0

print(classification_report(label_hate, pred_hate))

              precision    recall  f1-score   support

       False       0.94      0.70      0.80       582
        True       0.35      0.78      0.48       118

    accuracy                           0.71       700
   macro avg       0.64      0.74      0.64       700
weighted avg       0.84      0.71      0.75       700



In [ ]:
df_train[df_train["POLITICS"] > 0 ]

In [116]:
sample_df[pred_hate != label_hate][["context_tweet", "text", "text_label", "pred_cot", ]].sample(10)

,context_tweet,text,text_label,pred_cot
86,"Aborto legal: el Gobierno mantiene la idea de enviar el proyecto este año, pero no será hasta después de septiembre | Por Marcos Shaw\nhttps://t.co/kIMcSBkV70",@usuario Cómo les gusta que las usen a estas pelotudas,nada,"El texto menciona a las mujeres y las desprecia llamándolas ""pelotudas"". La respuesta final es ""mujer""."
170,La declaración de un testigo del caso del jubilado que mató a un ladrón: “Levantaba la mano pidiendo ayuda y el hombre le volvió a disparar” | Por Leonardo Scannone https://t.co/46a0CqmsaD,"@usuario El jubilado no sólo fue asaltado sino que fue golpeado salvajemente por los delincuentes. Ni me quiero imaginar el dolor, el miedo y el shock en el que estaba ese hombre. Me importa un bledo del delincuente. UNO MENOS.",criminal,"El texto no contiene discurso discriminatorio. La respuesta final es ""nada""."
584,"Macri, íntimo: qué piensa y cómo planea su reconstrucción política | Por María Julia Oliván https://t.co/7YhhX4uDbp",@usuario Feliz día del trabajo infantil! A explotar esas manitos!,nada,"El texto alude a la explotación laboral de los niños. La respuesta final es ""clase"""
149,El Polo Obrero realiza un corte en la Panamericana en contra de la flexibilización de la cuarentena y en reclamo de aumentos a los planes sociales https://t.co/fHOeCKJdY6,"@usuario A ver si entendí, cortan para q nadie vaya a laburar y q leaumente la limosna ??!!!",nada,"El texto denigra a las personas que están reclamando por aumentos a los planes sociales, llamándolos ""limosna"". La respuesta final es ""clase""."
487,"Robó a una embarazada, los vecinos lo lincharon y pidió clemencia: “Perdón, tengo dos pibes” https://t.co/gOcURtu6LT","@usuario y no entiendo, porqué no le quebraron las piernas o algo?",criminal,"El texto alude a causarle daño físico al ladrón por su acción de robar, lo cual puede ser considerado discurso discriminatorio. Sin embargo, no se mencionan características protegidas en el mensaje. La respuesta final es ""nada""."
653,"Lanata: ""Hoy Alberto está funcionando como el secretario de Cristina "" https://t.co/iFX8XgP0sm https://t.co/VGDfiVoUea",@usuario ES UN REBERENDO PELOTUDO FORRO DELA VIEJA CORRUPTA DE CUENTE MAFIOSA!!!,aspecto,"El texto contiene insultos y descalificaciones, pero no menciona ninguna de las características protegidas. La respuesta final es 'nada'."
348,"China prestará USD 1,000 millones a México, América Latina y el Caribe para la compra de vacunas contra el COVID-19 https://t.co/oxL59BpvNa","@usuario Son unos desgraciados los políticos chinos, unos miserables y rastreros. Para Ecuador intenta destruir la reputación del mejor camarón del mundo #iloveshrimpfromecuador, luego está el asecho a #Galapagos #SOSGALAPAGOS y ahora su chantaje con la vacuna. #ChinaLiedPeopleDied .",nada,"El texto denigra y desprecia a los políticos chinos, los acusa de ser miserables y rastreros. Además, utiliza el término ""desgraciados"". La respuesta final es ""política""."
288,Cristina Pérez vivió un incómodo momento en vivo con Sergio Berni: “No tiene necesidad de descalificarme” https://t.co/ZA45DZ0C05,"@usuario Lo que pasa es que esa chica es medio opa, es difícil que no se incomode: dice boludeces todo el tiempo, realmente me pregunto si es tonta o se hace.",nada,"El texto alude a la capacidad mental de Cristina Pérez, insinuando que podría ser ""opa"" o tonta. La respuesta final es ""discapacidad, aspecto""."
361,"Gerardo Morales volvió a diferenciarse de Mauricio Macri: ""No comparto su visión, hay que apoyar al Gobierno"" https://t.co/4DtZSoNdne",@usuario Ya van a ir x vos...,nada,"El texto parece ser una amenaza o advertencia hacia Gerardo Morales, indicando que habrá consecuencias o represalias contra él. La respuesta final es ""política""."
688,Aborto: miles de mujeres volvieron a las calles para reclamar que se trate y sea ley https://t.co/hbfjyaIYbA,@usuario Vayan a sus casas señoras! La economía del país se cae a pedazos y ustedes piden más gas

In [112]:
# LGBTI errors

label = "CLASS"
sample_df.loc[sample_df[label] != sample_df[f"PRED_{label}"]][["context_tweet", "text", "text_label", "pred_cot"]].sample(5)

,context_tweet,text,text_label,pred_cot
690,"En promedio, los beneficiarios de planes sociales cobrarán $40.000 en diciembre | Por Pablo Wende https://t.co/OaSD2ZmmHy",@usuario @usuario Les seguimos pagando el crédito de ANSES con el q se compraron el 50’ \nArroyo mismo lo dijo cuando empezó la cuareterna: -muchos tienen créditos y les llega descontado del plan”-\nPagamos sus vicios y sus “gustitos “,nada,"El texto se refiere a quienes cobran planes sociales como personas que gastan el dinero en vicios y gustos. La respuesta final es ""clase"""
45,"Robó a una embarazada, los vecinos lo lincharon y pidió clemencia: “Perdón, tengo dos pibes” https://t.co/gOcURtu6LT","@usuario Anda a vender pastelitos, pelotudo, si es cierto que tenes dos hijos.",nada,"El texto insulta y menosprecia al individuo llamándolo ""pelotudo"" y sugiriendo que venda pastelitos como forma de subsistencia. La respuesta final es ""clase"""
40,Yanina Latorre le respondió a quienes la cuestionaron porque su mamá se vacunó contra el COVID-19 en Estados Unidos https://t.co/w9t3G6NIBX,@usuario Ojalá todo esto genere que la familia Latorre en pleno se vaya a vivir al exterior. Son prescindibles,nada,"El comentario sugiere que la familia Latorre debería irse del país, lo cual implica que no son necesarios o no son importantes. La respuesta final es ""clase"""
426,Corrientes: cesárea programada para la nena de 10 años violada por la ex pareja de su mamá https://t.co/NOZxEpINON,"@usuario ""Fue seducida"" y ""tiene un leve trauma"" ese juez es una verdadera basura",nada,"El texto pone en duda la versión de la niña de 10 años que fue violada, culpando al juez y llamándolo basura. La respuesta final es ""clase, criminal"""
31,"Grabois calificó al campo como ""parásitos que viven de la renta extraordinaria de la tierra que es de todos"" https://t.co/c8XQr9c1aA","@usuario LA TIERRA ES PARA EL QUE LA COMPRO CON ESFUERZO Y SUDOR DE SANGRE, PARA QUE VENGAS VOS IMBECIL A QUERER DECIR QUE ES DE TODOS. NO SABES DIFERENCIAR UNA VACA DE UN CHIMANGO.",nada,"El texto alude a la condición de clase del destinatario del mensaje (campesinos), y lo desprecia llamándolo imbecil e insinuando que es ignorante. La respuesta final es ""clase"""


In [117]:
df_train[(df_train["POLITICS"] > 0) & (df_train["num_hatred"] > 1)][["context_tweet", "text", "text_label"]].sample(5)

,context_tweet,text,text_label
32287,"Cecilia Moreau: ""No se puede permitir que la minoría le imponga a la mayoría qué temas discutir"" https://t.co/DIOKystxnf",@usuario COCHINA MENTIROSA!! CÓMPLICE canalla de la mafia KAKA!!,"política, aspecto"
26017,Coronavirus en Argentina: el Gobierno convocará a miles de piqueteros para convencer a la gente que se vacune https://t.co/XofDz33SV1,@usuario Los quieren concencer a los palazos? Vacunena los piqueteros y pkanwros primero. @usuario @usuario @usuario @usuario @usuario,"clase, política"
7622,"""Día menstrual para estudiantes "": el proyecto de ley de Carmela Moreau https://t.co/OfxmKpX7bT https://t.co/YxTJbSrQ3k","@usuario Gobierno nacional popular y generador de excusas para no trabajar, y digo gobierno x que esta inepta, incapaz, indigna y descerebrada de Carmela tiene el sello kk ya incorporado en el adn #GobiernoDeInutiles #gobiernodemierda #GobiernoDeMierda","mujer, política"
28206,Santa Fe: rompen más silobolsas y crece la preocupación en el campo https://t.co/Pb1gaBqyXU https://t.co/cPlGjy3V6T,"@usuario hay que hacer guardia, meter bala y enterrar a los soretes para abonar la tierra","política, criminal"
28983,'País de maricas': ¿Por qué Jair Bolsonaro vuelve a atacar a los homosexuales en Brasil? https://t.co/pVdFjiztkm,"@usuario El discurso de bolsonaro donde dice ""maricas"" es brillante, sólo eso basta para meresca ser el presidente de Sudamérica. Clarín es una cueva de zurdos y progresistas que se pintan los labios punto. Clarín suavecito, te pasas de marica!","lgbti, política"
